In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import time

# Value Iteration

In [2]:
def value_iter(env, gamma, theta):
    # env.env.nS is the number of states
    # env.env.nA is the number of actions
    # env.env.P is the model that includes:
    # p: probability of going from one state to another via a specific action
    # r: reward
    # s_: next state
    V = np.zeros(env.env.nS) #Initial Values
    pi = np.zeros(env.env.nS) #Initial Policy
    count_iter = 0 # for Counting Iteration
    while True:
        count_iter+=1
        delta = 0
        for s in range(env.env.nS): #for each state compute:
            v = V[s]
            values = list()
            for a in range(env.env.nA): #for each action compute
                val = 0
                for p, s_, r, _ in env.env.P[s][a]:
                    val += p * (r + gamma * V[s_])#value that gets added if a specific action is taken in a specific state
                values.append(val)
            V[s] = max(values)#Figuring out the best value for this state
            pi[s] = np.argmax(values)#Figuring out which action gives the best value
            delta = max(delta, abs(v - V[s]))#Checking how much the value changes
        if delta < theta: break#If there's not much change in value, stop
    
    return V, pi, count_iter

In [3]:
#Defining the map
custom_map = [
    'SFFFH',
    'FFHHF',
    'FFFFF',
    'HHFHF',
    'FFFFG'
]
#Using gym to create the environment
env = gym.make('FrozenLake-v0', desc=custom_map, is_slippery=True)
#Resetting the game
env.reset()
env.render()


SFFFH
FFHHF
FFFFF
HHFHF
FFFFG


In [4]:
t0 = time.time()
V, pi, countiter = value_iter(env, gamma=0.85, theta=1e-4)
t1 = time.time()
print('Value Iteration')
print(V.reshape([5, 5]))
print('Time: ', t1-t0)
print('Number  of Iterations: ', countiter)
actdict = {0:'L', 1:'D', 2:'R', 3:'U'}
policy = np.array([actdict[p] for p in pi])
print('Policy:\n', np.array(policy).reshape([5, 5]))

Value Iteration
[[0.00698058 0.00750863 0.00350883 0.00138496 0.        ]
 [0.01021292 0.01204922 0.         0.         0.10575782]
 [0.01383751 0.02481905 0.06172147 0.09328092 0.26750555]
 [0.         0.         0.09976105 0.         0.57087423]
 [0.10099983 0.15470592 0.29041378 0.57992437 0.        ]]
Time:  0.03497815132141113
Number  of Iterations:  27
Policy:
 [['D' 'L' 'U' 'L' 'L']
 ['D' 'L' 'L' 'L' 'D']
 ['U' 'U' 'D' 'D' 'R']
 ['L' 'L' 'L' 'L' 'R']
 ['D' 'D' 'D' 'D' 'L']]


# Policy Iteration

In [5]:
def policy_iter(env, gamma, theta):
    # env.env.nS is the number of states
    # env.env.nA is the number of actions
    # env.env.P is the model that includes:
    # p: probability of going from one state to another via a specific action
    # r: reward
    # s_: next state
    V = np.zeros(env.env.nS) #Initial Values
    #pi = np.random.randint(4,size=env.env.nS) #Random Initial Policies
    pi = np.ones(env.env.nS) #Initial Policies (all of them are set to one)
    count_iter = 0# for Counting Iteration
    while True:
        count_iter+=1
        #Compute Values for this Policy
        while True:
            delta = 0
            for s in range(env.env.nS):#for each state compute:
                v = V[s]
                val = 0
                for p, s_, r, _ in env.env.P[s][pi[s]]:#compute only for action 1 (down)
                    val += p * (r + gamma * V[s_])
                V[s] = val
                delta = max(delta, abs(v - V[s]))#Checking how much the value changes
            if delta < theta: break#If there's not much change in value, stop
        #Update Policy
        policy_stable = True #Assuming we've already got the best policy
        for s in range(env.env.nS):#for each state compute:
            old_action = pi[s]
            values = list()
            for a in range(env.env.nA):#for each action compute:
                val = 0
                for p, s_, r, _ in env.env.P[s][a]:
                    val += p * (r + gamma * V[s_])#value that gets added if a specific action is taken in a specific state
                values.append(val)#Figuring out the best value for this state
            pi[s] = np.argmax(values)#Figuring out which action gives the best value
            if old_action != pi[s]: policy_stable = False #If It's not the old action keep updating
        if policy_stable: break#If the policy has stablized, stop updating
            
    return V, pi, count_iter

In [6]:
env.reset()
env.render()


SFFFH
FFHHF
FFFFF
HHFHF
FFFFG


In [7]:
t0 = time.time()
V, pi, countiter = policy_iter(env, gamma=0.85, theta=1e-4)
t1 = time.time()
print('Policy Iteration')
print(V.reshape([5, 5]))
print('Time: ', t1-t0)
print('Number  of Iterations: ', countiter)
actdict = {0:'L', 1:'D', 2:'R', 3:'U'}
policy = np.array([actdict[p] for p in pi])
print('Policy:\n', np.array(policy).reshape([5, 5]))

Policy Iteration
[[0.00688123 0.00742604 0.00345214 0.0013585  0.        ]
 [0.01015866 0.01200206 0.         0.         0.10575116]
 [0.01379926 0.02479848 0.06173264 0.09328062 0.26749831]
 [0.         0.         0.09980908 0.         0.5708703 ]
 [0.10128224 0.15490417 0.29053499 0.57997889 0.        ]]
Time:  0.019987821578979492
Number  of Iterations:  3
Policy:
 [['D' 'L' 'U' 'L' 'L']
 ['D' 'L' 'L' 'L' 'D']
 ['U' 'U' 'D' 'D' 'R']
 ['L' 'L' 'L' 'L' 'R']
 ['D' 'D' 'D' 'D' 'L']]
